In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.models import Sequential
import keras.backend as K

Using TensorFlow backend.


In [2]:
df = pd.read_csv("flower_photos.csv")
df.head()

,label,pixel0,pixel1,pixel10,pixel100,pixel1000,pixel1001,pixel1002,pixel1003,pixel1004,...,pixel9990,pixel9991,pixel9992,pixel9993,pixel9994,pixel9995,pixel9996,pixel9997,pixel9998,pixel9999
0,dandelion,31,32,32,32,39,39,38,37,36,...,26,27,27,27,27,27,27,27,28,27
1,dandelion,132,133,155,141,141,140,145,162,170,...,2,2,1,0,0,0,0,0,0,0
2,dandelion,22,85,32,49,32,42,39,38,53,...,52,59,32,74,38,34,54,51,44,45
3,dandelion,67,59,109,63,44,47,52,56,62,...,38,42,46,48,49,51,52,54,54,55
4,dandelion,203,192,173,192,175,182,173,195,182,...,167,183,186,199,189,187,190,184,199,197


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3670 entries, 0 to 3669
Columns: 10001 entries, label to pixel9999
dtypes: int64(10000), object(1)
memory usage: 280.0+ MB


In [4]:
onehot_enc = OneHotEncoder(sparse=False)
X = df.drop("label", axis=1).values / 255
y = onehot_enc.fit_transform(df["label"].values.reshape(-1, 1))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
model = Sequential()
model.add(Conv2D(8, kernel_size=9, activation="relu", input_shape=(100, 100, 1)))
model.add(Conv2D(16, kernel_size=9, activation="relu"))
model.add(MaxPooling2D())
model.add(Conv2D(32, kernel_size=5, activation="relu"))
model.add(Conv2D(64, kernel_size=5, activation="relu"))
model.add(MaxPooling2D())
model.add(Conv2D(96, kernel_size=3, activation="relu"))
model.add(Conv2D(128, kernel_size=3, activation="relu"))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.25))
model.add(Dense(32, activation="relu"))
model.add(Dense(5, activation="softmax"))

model.summary()

W0829 12:50:27.987769 139770894141248 deprecation_wrapper.py:119] From /home/tustunkok/miniconda3/envs/dl/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0829 12:50:27.999593 139770894141248 deprecation_wrapper.py:119] From /home/tustunkok/miniconda3/envs/dl/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0829 12:50:28.001130 139770894141248 deprecation_wrapper.py:119] From /home/tustunkok/miniconda3/envs/dl/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0829 12:50:28.037793 139770894141248 deprecation_wrapper.py:119] From /home/tustunkok/miniconda3/envs/dl/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecate

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 92, 92, 8)         656       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 84, 84, 16)        10384     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 42, 42, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 38, 38, 32)        12832     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 34, 34, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 15, 15, 96)        55392     
__________

In [7]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

W0829 12:50:53.811442 139770894141248 deprecation_wrapper.py:119] From /home/tustunkok/miniconda3/envs/dl/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0829 12:50:53.840892 139770894141248 deprecation_wrapper.py:119] From /home/tustunkok/miniconda3/envs/dl/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [8]:
history = model.fit(X_train.reshape(-1, 100, 100, 1), y_train, batch_size=128, epochs=30, validation_data=(X_test.reshape(-1, 100, 100, 1), y_test))

W0829 12:50:55.033450 139770894141248 deprecation.py:323] From /home/tustunkok/miniconda3/envs/dl/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 2936 samples, validate on 734 samples
Epoch 1/30
2936/2936 [==============================] - 63s 22ms/step - loss: 1.6074 - acc: 0.2258 - val_loss: 1.6022 - val_acc: 0.2725
Epoch 2/30
2936/2936 [==============================] - 65s 22ms/step - loss: 1.6033 - acc: 0.2371 - val_loss: 1.5969 - val_acc: 0.2725
Epoch 3/30
2936/2936 [==============================] - 65s 22ms/step - loss: 1.5984 - acc: 0.2340 - val_loss: 1.5958 - val_acc: 0.2725
Epoch 4/30
2936/2936 [==============================] - 65s 22ms/step - loss: 1.6038 - acc: 0.2384 - val_loss: 1.5963 - val_acc: 0.2725
Epoch 5/30
2688/2936 [==========================>...] - ETA: 5s - loss: 1.5945 - acc: 0.2325

KeyboardInterrupt: 

In [30]:
predictions = model.predict(X_test.reshape(-1, 100, 100, 1))
predictions = onehot_enc.inverse_transform(predictions)
y_test = onehot_enc.inverse_transform(y_test)

In [31]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

       daisy       0.39      0.54      0.46       129
   dandelion       0.55      0.44      0.49       200
       roses       0.38      0.25      0.30       126
  sunflowers       0.37      0.31      0.34       127
      tulips       0.38      0.51      0.44       152

    accuracy                           0.42       734
   macro avg       0.42      0.41      0.41       734
weighted avg       0.43      0.42      0.42       734

